# CHAPTER 10 - Recommendation Engines
##Creating a Movies Recommendation Engine
Let's build a recommendation engine that can recommend movies to a bunch of users. 


## 1) Import libraries
Let us import the packages

In [3]:
import pandas as pd 
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
warnings.filterwarnings('ignore')

## 2) Data exploration

Let us import the user.csv

Let us use the value_counts() function is used to get uniques users and their count of reviews. Following code shows that user 414 has written 2698 reviews

In [4]:
df_reviews = pd.read_csv('reviews.csv')
df_reviews.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Let us count the number of reviews that we have.

In [5]:
df_reviews.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [6]:
df_reviews['userId'].value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
610    1302
68     1260
380    1218
606    1115
288    1055
249    1046
387    1027
182     977
307     975
603     943
298     939
177     904
318     879
232     862
480     836
608     831
600     763
590     728
483     728
105     722
19      703
305     677
489     648
111     646
438     635
       ... 
467      22
245      21
293      21
37       21
439      21
324      21
507      21
364      21
598      21
49       21
157      21
547      21
87       21
549      21
26       21
281      21
320      20
207      20
576      20
194      20
189      20
257      20
147      20
53       20
278      20
406      20
595      20
569      20
431      20
442      20
Name: userId, Length: 610, dtype: int64

## Now let us import the movie file 

In [7]:
df_movie_titles = pd.read_csv('movies.csv',index_col=False)
df_movie_titles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


We merge the two dataframes by the movie ID

In [8]:
df = pd.merge(df_reviews, df_movie_titles, on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


With the pandas "describe", it is possible to see the different columns and their statistics.

In [9]:
df.describe()

,userId,movieId,rating,timestamp
count,98180.000000,98180.000000,98180.000000,9.818000e+04
mean,326.348747,15938.331259,3.502567,1.197962e+09
std,182.344624,28668.177407,1.040068,2.135274e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1193.000000,3.000000,1.008558e+09
50%,325.000000,2890.000000,3.500000,1.179177e+09
75%,477.000000,7121.000000,4.000000,1.428575e+09
max,610.000000,131237.000000,5.000000,1.537799e+09


In order to understand the significance of the ratings, I calculate the mean and count of ratings per movie. In this sense, we perform a "groupby" by the columns "title" and "rating".

In [10]:
df_ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
df_ratings['number_of_ratings'] = df.groupby('title')['rating'].count()
df_ratings.head()

,rating,number_of_ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


# 3) Converting the data from narrow to wide format

In order to do create the item-based recommender system we need to convert our dataset into a matrix with the movie titles as the columns, the user_id as the index and the ratings as the values. By doing this we shall get a dataframe with the columns as the movie titles and the rows as the user ids. Each column represents all the ratings of a movie by all users. The rating appear as NAN where a user didn't rate a certain movie. We shall use this matrix to compute the correlation between the ratings of a single movie and the rest of the movies in the matrix. 

In [11]:
movie_matrix = df.pivot_table(index='userId', columns='title', values='rating')
movie_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),"...All the Marbles (California Dolls, The) (1981)",...,Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let’s assume that a user has watched Avatar (2009) and Thor (2011). We would like like to recommend movies to this user based on this watching history. The goal is to look for movies that are similar to Thor (2011) and Avatar (2009) which we shall recommend to this user. We can achieve this by computing the correlation between these two movies’ ratings and the ratings of the rest of the movies in the dataset.

In [12]:
Avatar_user_rating = movie_matrix['Avatar (2009)']
Avatar_user_rating = Avatar_user_rating.dropna()
Avatar_user_rating.head()

userId
10    2.5
15    3.0
18    4.0
21    4.0
22    3.5
Name: Avatar (2009), dtype: float64

In order to compute the correlation between two dataframes we use pandas corrwith functionality. Corrwith computes the pairwise correlation of rows or columns of two dataframe objects. Let's use this functionality to get the correlation between each movie's rating and the ratings of the Avatar movie.

In [13]:
similar_to_Avatar=movie_matrix.corrwith(Avatar_user_rating)
corr_Avatar = pd.DataFrame(similar_to_Avatar, columns=['correlation'])
corr_Avatar.dropna(inplace=True)
corr_Avatar = corr_Avatar.join(df_ratings['number_of_ratings'])
corr_Avatar.head()

,correlation,number_of_ratings
title,,
"'burbs, The (1989)",0.353553,17
(500) Days of Summer (2009),0.131120,42
*batteries not included (1987),0.785714,7
10 Things I Hate About You (1999),0.265637,54
"10,000 BC (2008)",-0.075431,17


Some of the movies have very few ratings and may end up being recommended simply because one or two people gave them a 5 star rating. We can fix this by setting a threshold for the number of ratings. From the histogram earlier we saw a sharp decline in number of ratings from 100. We shall therefore set this as the threshold. In order to do this we need to join the two dataframes with the number_of_ratings column in the ratings dataframe.

In [14]:
corr_Avatar[df_ratings['number_of_ratings'] > 100].sort_values(by='correlation', ascending=False).head(10)

,correlation,number_of_ratings
title,,
Trainspotting (1996),0.623735,102
"Monsters, Inc. (2001)",0.541616,132
"Net, The (1995)",0.530552,112
Finding Nemo (2003),0.501461,141
"Matrix, The (1999)",0.493241,278
Die Hard: With a Vengeance (1995),0.466794,144
Men in Black (a.k.a. MIB) (1997),0.459441,165
Star Wars: Episode VI - Return of the Jedi (1983),0.458785,196
"Terminator, The (1984)",0.448443,131


Finally, we have the list of movies that have similar ratings from the users than Avatar.
We perform the same for the Thor (2011) movie.

In [15]:
Thor_user_rating = movie_matrix['Thor (2011)']
Thor_user_rating = Thor_user_rating.dropna()
Thor_user_rating

userId
18     3.5
62     4.0
68     3.5
73     5.0
106    4.0
125    3.0
139    2.0
177    4.0
184    3.0
246    4.0
249    4.0
274    2.0
291    4.0
292    4.0
298    3.5
332    3.5
339    2.5
344    4.0
380    5.0
414    3.0
432    4.0
448    2.5
475    5.0
489    3.0
509    3.5
525    3.0
534    3.5
550    4.0
561    4.0
567    1.5
573    4.5
596    3.5
599    2.5
610    3.5
Name: Thor (2011), dtype: float64

In [16]:
similar_to_Thor=movie_matrix.corrwith(Thor_user_rating)
similar_to_Thor

title
'71 (2014)                                                                                       NaN
'Hellboy': The Seeds of Creation (2004)                                                          NaN
'Round Midnight (1986)                                                                           NaN
'Salem's Lot (2004)                                                                              NaN
'Til There Was You (1997)                                                                        NaN
'burbs, The (1989)                                                                          0.997176
'night Mother (1986)                                                                             NaN
(500) Days of Summer (2009)                                                                 0.386820
*batteries not included (1987)                                                                   NaN
...All the Marbles (California Dolls, The) (1981)                                    

In [17]:
corr_Thor = pd.DataFrame(similar_to_Thor, columns=['correlation'])
corr_Thor.dropna(inplace=True)
corr_Thor = corr_Thor.join(df_ratings['number_of_ratings'])
corr_Thor.head()

,correlation,number_of_ratings
title,,
"'burbs, The (1989)",0.997176,17
(500) Days of Summer (2009),0.386820,42
10 Things I Hate About You (1999),0.273560,54
"10,000 BC (2008)",0.245718,17
101 Dalmatians (1996),0.089730,47


In [18]:
corr_Thor[df_ratings['number_of_ratings'] > 100].sort_values(by='correlation', ascending=False).head(10)

,correlation,number_of_ratings
title,,
Stargate (1994),0.759494,140
Aladdin (1992),0.730377,183
Jurassic Park (1993),0.714481,238
"Lion King, The (1994)",0.710244,172
"Bourne Identity, The (2002)",0.705183,112
X-Men (2000),0.670755,133
Up (2009),0.665904,105
Batman Begins (2005),0.665208,116
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.658109,107
